In [165]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [185]:
import logging
import unittest
import colorama
import pandas as pd
import numpy as np

from communication.message.MessagePerformative import MessagePerformative
from communication.preferences.CriterionName import CriterionName
from communication.preferences.CriterionValue import CriterionValue
from communication.preferences.Item import Item
from communication.preferences.Preferences import Preferences
from communication.preferences.Value import Value
from pw_argumentation import ArgumentModel, format_argument

colorama.init()  # INFO: used to print colored text on Windows

## An example conversation between 2 agents

### Performance Tables

In [176]:
# generate random preferences
def generate_pref_df(n_items=2, n_crit=len(CriterionName), n_values=len(Value)):
    # generate preferences
    df = pd.DataFrame(np.random.randint(0, n_values, size=(n_items, n_crit)))
    # shuffle agent criteria preference order
    df.columns = np.random.permutation(df.columns)
    # drop random number of preferences
    df = df.drop(columns=np.random.choice(df.columns, size=np.random.randint(0, len(df.columns)), replace=False))
    return df

def generate_preferences():
    df = generate_pref_df()
    for i, row in df.iterrows():
        list_criteria = []
        criteria_values = []
        for j, val in row.iteritems():
            list_criteria.append(CriterionName(j))
            criteria_values.append(CriterionValue(Item(f"item{i+1}"), CriterionName(j), Value(val)))
            df.rename(columns={j: CriterionName(j).name}, inplace=True)
        df.rename(index={i: Item(f"item{i+1}")}, inplace=True)
    return Preferences(list_criteria, criteria_values), df

agents_prefs = []  # list of agents agents_prefs

Agent 1:

In [179]:
prefs, df = generate_preferences()
agents_prefs.append(Preferences(list_criteria, criteria_values))
print("Agent 1 preferences:")
df

Agent 1 preferences:


,PRODUCTION_COST,ENVIRONMENT_IMPACT,NOISE
item1,1,2,1
item2,0,4,4


Agent 2:

In [181]:
df = generate_pref_df()
for i, row in df.iterrows():
    list_criteria = []
    criteria_values = []
    for j, val in row.iteritems():
        list_criteria.append(CriterionName(j))
        criteria_values.append(CriterionValue(Item(f"item{i+1}"), CriterionName(j), Value(val)))
        df.rename(columns={j: CriterionName(j).name}, inplace=True)
    df.rename(index={i: Item(f"item{i+1}")}, inplace=True)
agents_prefs.append(Preferences(list_criteria, criteria_values))
print("Agent 2 preferences:")
df

Agent 2 preferences:


,DURABILITY,CONSUMPTION,ENVIRONMENT_IMPACT,PRODUCTION_COST,NOISE
item1,3,1,1,2,2
item2,4,2,3,4,3


In [183]:
results

{'winning_agent': 'A5',
 'winning_item': item2,
 'winning_argument': 'top_10_percent'}

In [186]:
# argument_model = ArgumentModel(agents_prefs)
# argument_model.run_model()

# results, history = argument_model.get_final_result()
if results:
    print("Winning agent:", results["winning_agent"])
    print("Winning item:", results["winning_item"])
    print("Winning argument:", format_argument(results["winning_argument"]))

history

Winning agent: A5
Winning item: item2


TypeError: string indices must be integers

## Start negotiations

## Statistics